<a href="https://colab.research.google.com/github/jakobbaumgartner/CNN-HotDog-Classifier/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
# %tensorflow_version 1.x

In [26]:
import tensorflow as tf

In [27]:
tf.keras.__version__

'2.4.0'

In [28]:
import numpy as np
import matplotlib.pyplot as plt
import os
import cv2
from tqdm import tqdm

In [ ]:
Če imam izbrano GPU:

In [29]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [ ]:
Če pa bi želel uporabiti TPU: 

In [30]:
# use_tpu = True #@param {type:"boolean"}

# if use_tpu:
#     assert 'COLAB_TPU_ADDR' in os.environ, 'Missing TPU; did you request a TPU in Notebook Settings?'

# if 'COLAB_TPU_ADDR' in os.environ:
#   TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])
# else:
#   TF_MASTER=''

In [31]:
# resolver = tf.distribute.cluster_resolver.TPUClusterResolver(TF_MASTER)
# tf.config.experimental_connect_to_cluster(resolver)
# tf.tpu.experimental.initialize_tpu_system(resolver)
# strategy = tf.distribute.experimental.TPUStrategy(resolver)

Nastavimo poti do fotografij, uredimo fotografije. In izdelamo training dataset.

In [32]:
categories = ["hot_dog", "not_hot_dog"]
datadir = "drive/MyDrive/Colab Notebooks/seefood/train"
os.getcwd()
IMG_SIZE = 400 # maybe change this setting later, if the results are not good

training_data = []

for category in categories:  # do hot_dog and not_hot_dog

    path = os.path.join(datadir,category)  # create path to hotdogs
    class_num = categories.index(category)  # get the classification  (0 or a 1). hot or not

    for img in tqdm(os.listdir(path)):  # iterate over each image 
        try:
            img_array = cv2.imread(os.path.join(path,img) ,cv2.IMREAD_GRAYSCALE)  # convert to grayscale array
            new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE)) # resize 
            training_data.append([new_array, class_num]) 
          
        except Exception as e:  # in the interest in keeping the output clean...
              print(e)
      
import random
random.shuffle(training_data)

X = []   # features (big X)
y = []   # labels (small y)

for features,label in training_data:
    X.append(features) 
    y.append(label)   # labels

print(X[0].reshape(-1, IMG_SIZE, IMG_SIZE, 1))

# we have to make features into a np array
X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1) # number of features, size_x, size_y, colors (grayscale = 1)

100%|██████████| 449/449 [00:02<00:00, 205.78it/s]


[[[[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  [[0]
   [0]
   [0]
   ...
   [0]
   [0]
   [0]]

  ...

  [[1]
   [1]
   [2]
   ...
   [0]
   [0]
   [0]]

  [[1]
   [1]
   [2]
   ...
   [0]
   [0]
   [0]]

  [[1]
   [1]
   [2]
   ...
   [0]
   [0]
   [0]]]]


Sedaj želimo premešati podatke, da niso vse slike kategorij po vrsti, drugače imamo težave z učenjem.

In [33]:
import pickle

pickle_out = open("X.pickle","wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle","wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in = open("X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("y.pickle","rb")
y = pickle.load(pickle_in)

 *The basic CNN structure is as follows: Convolution -> Pooling -> Convolution -> Pooling -> Fully Connected Layer -> Output*

*Convolution is the act of taking the original data, and creating feature maps from it.Pooling is down-sampling, most often in the form of "max-pooling," where we select a region, and then take the maximum value in that region, and that becomes the new value for the entire region. Fully Connected Layers are typical neural networks, where all nodes are "fully connected." The convolutional layers are not fully connected like a traditional neural network.* 

Na tej točki začnemo zares načrtovati nevronsko mrežo.

In [34]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import TensorBoard

%load_ext tensorboard

import time

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [35]:
X=np.array(X/255.0)
y=np.array(y)


In [36]:
!pwd

/content


In [ ]:
# NAME = "hot_dog_CNN"
# tensorboard_callback = TensorBoard(log_dir="logs/{}".format(NAME))

# # log_dir = '/gdrive/My Drive/project/' + "logs/fit/"
# # tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

# model = Sequential()

# model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(128, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))


# model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

# model.add(Dense(64))
# model.add(Activation('relu'))

# model.add(Dense(1))
# model.add(Activation('sigmoid'))

# model.compile(loss='binary_crossentropy',
#               optimizer='adam',
#               metrics=['accuracy'])

# model.fit(X, y, 
#           batch_size=8, 
#           epochs=16,
#           verbose = 1, 
#           callbacks=[tensorboard_callback],
#           validation_split=0.1
#           )

Sedaj treniramo model in spreminjamo parametre, da se poizkusimo izogniti overfiting.

In [ ]:
NAME = "hot_dog_CNN"
tensorboard_callback = TensorBoard(log_dir="logs/{}".format(NAME))

# log_dir = '/gdrive/My Drive/project/' + "logs/fit/"
# tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model = Sequential()

model.add(Conv2D(256, (3, 3), input_shape=X.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(64))
model.add(Activation('relu'))

model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

model.fit(X, y, 
          batch_size=8, 
          epochs=16,
          verbose = 1, 
          callbacks=[tensorboard_callback],
          validation_split=0.1
          )

In [ ]:
%tensorboard --logdir logs  # show tensorBoard 